<a href="https://colab.research.google.com/github/kushal-h/CNN_models/blob/master/CNN_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=271f19d529b021ae8bccf7ef56cbb97767486876df847fa7ac34434320a3160f
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=bec400c033df2ae97598808353ff48b06345e9ca45c12e5a9cf8ef2879241d31
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np

In [0]:
fashion_mnist= keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
train_images= train_images/255.0
test_images= test_images/255.0

In [6]:
train_images[0].shape 

(28, 28)

In [0]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5, directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 199s 118ms/step - loss: 0.3918 - accuracy: 0.8586 - val_loss: 0.3068 - val_accuracy: 0.8803
Epoch 2/3
1688/1688 [==============================] - 199s 118ms/step - loss: 0.2614 - accuracy: 0.9025 - val_loss: 0.2687 - val_accuracy: 0.9020
Epoch 3/3
1688/1688 [==============================] - 198s 117ms/step - loss: 0.2019 - accuracy: 0.9242 - val_loss: 0.2636 - val_accuracy: 0.9055


Epoch 1/3
1688/1688 [==============================] - 128s 76ms/step - loss: 0.3922 - accuracy: 0.8605 - val_loss: 0.2928 - val_accuracy: 0.8937
Epoch 2/3
1688/1688 [==============================] - 127s 75ms/step - loss: 0.2514 - accuracy: 0.9093 - val_loss: 0.2740 - val_accuracy: 0.9040
Epoch 3/3
1688/1688 [==============================] - 126s 75ms/step - loss: 0.1942 - accuracy: 0.9299 - val_loss: 0.2517 - val_accuracy: 0.9100


Epoch 1/3
1688/1688 [==============================] - 477s 283ms/step - loss: 0.4100 - accuracy: 0.8517 - val_loss: 0.3239 - val_accuracy: 0.8730
Epoch 2/3
1688/1688 [==============================] - 475s 281ms/step - loss: 0.2736 - accuracy: 0.8990 - val_loss: 0.2764 - val_accuracy: 0.8973
Epoch 3/3
1688/1688 [==============================] - 478s 283ms/step - loss: 0.2188 - accuracy: 0.9190 - val_loss: 0.2597 - val_accuracy: 0.9073


Epoch 1/3
1688/1688 [==============================] - 344s 204ms/step - loss: 0.4272 - accuracy: 0.8456 - val_loss: 0.3094 - val_accuracy: 0.8818
Epoch 2/3
1688/1688 [==============================] - 344s 204ms/step - loss: 0.2812 - accuracy: 0.8953 - val_loss: 0.2776 - val_accuracy: 0.8970
Epoch 3/3
 258/1688 [===>..........................]Buffered data was truncated after reaching the output size limit.

In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        13872     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,341,834
Trainable params: 1,341,834
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 130s 77ms/step - loss: 0.0139 - accuracy: 0.9950 - val_loss: 0.7649 - val_accuracy: 0.9118
Epoch 5/10
1688/1688 [==============================] - 128s 76ms/step - loss: 0.0124 - accuracy: 0.9960 - val_loss: 0.7505 - val_accuracy: 0.9102
Epoch 6/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0118 - accuracy: 0.9960 - val_loss: 0.7721 - val_accuracy: 0.9122
Epoch 7/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0122 - accuracy: 0.9957 - val_loss: 0.7659 - val_accuracy: 0.9012
Epoch 8/10
1688/1688 [==============================] - 130s 77ms/step - loss: 0.0118 - accuracy: 0.9961 - val_loss: 0.8261 - val_accuracy: 0.9045
Epoch 9/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0109 - accuracy: 0.9965 - val_loss: 0.8268 - val_accuracy: 0.9070
Epoch 10/10
1688/1688 [==============================] - 127s 75ms/step - loss: 0.0114 - accuracy: 0.9965 - val_loss: 